import all functions from file

In [1]:
from tool.functions import *

In [2]:
download_all_bidixes()

2018-06-11 22:02:17,226 | INFO : Start
2018-06-11 22:13:57,475 | INFO : Finish


For now I delete ces-ces, ita-ita, eus-eu_bis, hbs_HR-hbs_SR, zh_CN-zh_TW

In [2]:
%time monodix()

2018-06-12 16:41:10,577 | INFO : started
2018-06-12 16:44:19,892 | INFO : finished
Wall time: 3min 9s


In [2]:
%time get_relevant_languages('spa', 'eng')

Wall time: 44 ms


In [1]:
from tool.functions import load_file, change_encoding, built_from_file

In [2]:
%time load_file('spa', 'eng')

2018-06-11 22:32:06,560 | INFO : cat-ita started
2018-06-11 22:32:39,285 | INFO : eng-cat started
2018-06-11 22:37:19,892 | INFO : eng-ita started
2018-06-11 22:38:54,490 | INFO : eng-spa started
2018-06-11 22:41:22,105 | INFO : fra-cat started
2018-06-11 22:45:20,731 | INFO : fra-eng started
2018-06-11 22:46:16,861 | INFO : fra-ita started
2018-06-11 22:46:48,962 | INFO : fra-spa started
2018-06-11 22:48:43,558 | INFO : spa-cat started
2018-06-11 22:53:54,753 | INFO : spa-ita started
Wall time: 22min 55s


In [3]:
%time change_encoding('spa-eng')

Wall time: 296 ms


In [2]:
%time G = built_from_file('spa-eng')
%time l1, l2 = dictionaries('spa', 'eng')
'Edges: {}  Nodes: {}'.format(len(G.edges()),len(G.nodes()))

Wall time: 26.9 s
Wall time: 6.02 s


'Edges: 538634  Nodes: 285935'

** Search **

In [69]:
%timeit s = possible_translations(G, Word('be','eng',[Tags(['vblex']),Tags(['vblex','pp'])]), 'spa', cutoff=8, n=40)
%timeit s = nx.single_source_shortest_path_length(G, Word('be','eng',[Tags(['vblex']),Tags(['vblex','pp'])]),cutoff=8)

2.56 ms ± 156 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
352 ms ± 38.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [2]:
def evaluation(G, word, candidates, metric, cutoff=4):
    result = {}
    for translation in candidates:
        result[translation] = metric(G, word, translation, cutoff=4)
    return result

def sorting(result, n):
    result = [(x, result[x]) for x in sorted(result, key=result.get, reverse=True)[:n]]
    return result

def metric(G, word, translation, cutoff, mode='exp'):
    coef = 0
    if mode in ('exp'):
        t = Counter([len(i) for i in nx.all_simple_paths(G, word, translation, cutoff=cutoff)])
        if mode == 'exp':
            for i in t: 
                coef += exp(-t[i])
            return coef

In [3]:
def lemma_search (G, lemma, d_l1, l2, cutoff, n):
    lemmas = [i for i in d_l1.lemma(lemma) if i in G.nodes()]
    results = {word:{} for word in lemmas}
    for word in lemmas:
        candidates = possible_translations(G, word, l2, cutoff=cutoff, n=n)
        results[word] = evaluation(G, word, candidates, metric, cutoff=cutoff)
        del candidates
    return results

In [4]:
def print_lemma_results(results, n=10):
    for i in d:
        print ('\t\t', i)
        words = sorting(d[i], n)
        for j in words:
            print ('{}\t{}'.format(j[0], j[1]))
        print()

In [165]:
%time print_lemma_results(lemma_search (G, 'star', l2, 'spa', 4, 40), n = 5)

		 eng$star$[n_n-ND]
spa$astro$[n_n-m]	0.6385500076446677
spa$cometa$[n-m_n]	0.4176665095393063
spa$cometa$[n-f_n_n-f-sg]	0.4176665095393063
spa$estrella$[n-f_n_n-f-sg]	0.37461750070570254
spa$barrilete$[n-m_n]	0.36787944117144233

		 eng$star$[vblex]
spa$protagonizar$[vblex]	1.103638323514327
spa$jugar$[vblex_vblex-vbact]	0.13567074586451522
spa$obrar$[vblex]	0.1353352832366127
spa$hacer$[vblex_vblex-inf]	0.1353352832366127
spa$alivianarse$[vblex-pron]	0.1353352832366127

Wall time: 123 ms


In [166]:
%time s = lemma_search (G, 'capital', l1, 'eng', 4, 40)
%time s = lemma_search (G, 'capital', l1, 'eng', 6, 40)
%time s = lemma_search (G, 'capital', l1, 'eng', 8, 40)

Wall time: 122 ms
Wall time: 84.3 ms
Wall time: 78.1 ms


In [167]:
%time s = lemma_search (G, 'casa', l1, 'eng', 4, 40)
%time s = lemma_search (G, 'casa', l1, 'eng', 6, 40)
%time s = lemma_search (G, 'casa', l1, 'eng', 8, 40)

Wall time: 117 ms
Wall time: 78.1 ms
Wall time: 93.7 ms


** Evaluate on rus-eng **

In [ ]:
%time l1, l2 = dictionaries('rus', 'eng')
%time G = built_from_file('rus-eng')

Wall time: 4.05 s


In [5]:
import random
def get_evaluation_pairs(G, dictionary, target, n=500):
    k = 4
    pairs = []
    while len(pairs) < n:
        candidates = random.sample(dictionary, k*n)
        pairs = []
        for i in candidates:
            if i in G.nodes():
                s = FilteredList(list(G.neighbors(i))).lang(target)
                if len(s) == 1: pairs.append((i, s[0], n))
        print (k*n, len(pairs))
        k+=1
    return pairs[:n]

In [104]:
%time s = get_evaluation_pairs(G, l1, 'eng', n=1000)

500 105
1000 240
1500 314
2000 441
2500 541
Wall time: 153 ms


In [6]:
def node_search(G, node, lang, cutoff):
    if node not in G.nodes():
        return None
    results = {}
    #for translation in possible_translations(G, node, lang, cutoff, n = 30):
    candidates = possible_translations(G, word, l2, cutoff=cutoff, n=30)
    results[word] = evaluation(G, word, candidates, metric, cutoff=cutoff)
    return list(sorted(results, key=results.get, reverse=True))[:20]

def two_node_search (G, node1, node2, l1, l2, cutoff):
    if (node1, node2) in G.edges(): G.remove_edge(node1, node2)
    if (node2, node1) in G.edges(): G.remove_edge(node2, node1)
    res1 = node_search(G, node1, l2, cutoff)
    res2 = node_search(G, node2, l1, cutoff)
    coefficient = 0
    if node2 in res1: coefficient += 0.5*(len(res1) - res1.index(node2))/len(res1)
    if node1 in res2: coefficient += 0.5*(len(res2) - res2.index(node1))/len(res2) 
    return coefficient

def evaluate_graph(G, pairs, l1, l2, cutoff):
    result = []
    for i in pairs:
        result.append(two_node_search (G, i[0], i[1], l1, l2, cutoff))
    return result

In [12]:
%time result = evaluate_graph(G, pairs2, 'rus', 'eng', 4)

Wall time: 1h 14min 23s


In [13]:
sum(result)/10

38.735399926879445